# Generating Melodies with Melody RNN
## Douglas Eck
This notebook shows you how to generate new melodies from a trained Melody RNN model from a priming sequence. You'll see how to download bundles containing pre-trained models, generate priming sequences, instantiate and initialize models and generate new melodies from the models. The notebook also shows some hyperparameters useful for controlling generation, such as ``temperature``.

In [1]:
import os
import urllib2

from magenta.interfaces.midi import magenta_midi
from magenta.models.melody_rnn import melody_rnn_model
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.protobuf import generator_pb2

import magenta.music as mm

# Constants.
MODELS = ['basic_rnn', 'lookback_rnn', 'attention_rnn', 'performance']
BUNDLES = {}
BUNDLE_DIR = '/tmp/bundles'
if not os.path.exists(BUNDLE_DIR):
    os.makedirs(BUNDLE_DIR)
for model in MODELS:
    BUNDLES[model] = os.path.join(BUNDLE_DIR, '%s.mag' % model)

In [2]:
# Download model bundles.
def download_bundles(force_reload=False):
    for model, bundle_target in BUNDLES.iteritems():
        if not os.path.exists(bundle_target) or force_reload:
            response = urllib2.urlopen('http://download.magenta.tensorflow.org/models/%s' % ('%s.mag' % model))
            data = response.read()
            local_file = open(bundle_target, 'wb')
            local_file.write(data)
            local_file.close()
            print 'Bundle file %s downloaded.' % bundle_target
        else:
            print 'Bundle file %s exists already. Not downloading.' % bundle_target
download_bundles()

Bundle file /tmp/bundles/basic_rnn.mag exists already. Not downloading.
Bundle file /tmp/bundles/performance.mag exists already. Not downloading.
Bundle file /tmp/bundles/attention_rnn.mag exists already. Not downloading.
Bundle file /tmp/bundles/lookback_rnn.mag exists already. Not downloading.


In [3]:
# Create a primer sequence, plot it and play it.
primer = mm.Melody([60, 60, 67, 67, mm.MELODY_NO_EVENT, mm.MELODY_NO_EVENT, 
                    mm.MELODY_NOTE_OFF, mm.MELODY_NOTE_OFF],
                    steps_per_quarter=2)
mm.plot_sequence(primer.to_sequence())
mm.play_sequence(primer.to_sequence())

Loading BokehJS ...

In [4]:
# Generate some music from the attention_rnn model. You can also drop in 'lookback_rnn' or 'basic_rnn'.
model = 'performance'
generator = magenta_midi._load_generator_from_bundle_file(BUNDLES[model])


#config = melody_rnn_model.default_configs[model]
#generator = melody_rnn_sequence_generator.MelodyRnnSequenceGenerator(
#    model=melody_rnn_model.MelodyRnnModel(config),
#    details=config.details,
#    steps_per_quarter=config.steps_per_quarter,
#    bundle=mm.read_bundle_file(os.path.join(BUNDLE_DIR, '%s.mag' % model)))                                                     

# Generating a sequence looks more complicated than it is. One reason is that
# the generator interface is flexible enough for real-time interaction but also 
# requires a bit more work to generate just one melody.
generator_options = generator_pb2.GeneratorOptions()

# Timing and duration.
qpm = 140.0

INFO:tensorflow:Restoring parameters from /var/folders/j8/pf5rk0s90slf8l2kwsgbbjpm003dpx/T/tmpzyBkUM/model.ckpt
Loaded 'performance' generator bundle from file '/tmp/bundles/performance.mag'.


In [5]:
# Generate some music from the attention_rnn model. You can also drop in 'lookback_rnn' or 'basic_rnn'.
model = 'performance'
generator = magenta_midi._load_generator_from_bundle_file(BUNDLES[model])


#config = melody_rnn_model.default_configs[model]
#generator = melody_rnn_sequence_generator.MelodyRnnSequenceGenerator(
#    model=melody_rnn_model.MelodyRnnModel(config),
#    details=config.details,
#    steps_per_quarter=config.steps_per_quarter,
#    bundle=mm.read_bundle_file(os.path.join(BUNDLE_DIR, '%s.mag' % model)))                                                     

# Generating a sequence looks more complicated than it is. One reason is that
# the generator interface is flexible enough for real-time interaction but also 
# requires a bit more work to generate just one melody.
generator_options = generator_pb2.GeneratorOptions()

# Timing and duration.
#qpm = 140.0
#num_steps = 120
#seconds_per_step = 60.0 / qpm / generator.steps_per_quarter
#total_seconds = num_steps * seconds_per_step

# Temperature: Higher yields more random music; 1.0 is default. 
generator_options.args['temperature'].float_value = 1.0

# You can play with these parameters as well. 
# generator_options.args['beam_size'].int_value = 4
# generator_options.args['branch_factor'].int_value = 1

primer_sequence = primer.to_sequence(qpm=qpm)
generate_section = generator_options.generate_sections.add(
        start_time=primer_sequence.notes[-1].end_time,# + seconds_per_step,
        end_time=30)#total_seconds)
sequence = generator.generate(primer_sequence, generator_options)

# Play and view this masterpiece.
mm.plot_sequence(sequence)
mm.play_sequence(sequence, mm.midi_synth.fluidsynth)

INFO:tensorflow:Restoring parameters from /var/folders/j8/pf5rk0s90slf8l2kwsgbbjpm003dpx/T/tmpkaK2fn/model.ckpt
Loaded 'performance' generator bundle from file '/tmp/bundles/performance.mag'.
INFO:tensorflow:Need to generate 2871 more steps for this sequence, will try asking for 1160 RNN steps
INFO:tensorflow:Beam search yields sequence with log-likelihood: -2709.675573 


Loading BokehJS ...

ImportError: fluidsynth() was called but pyfluidsynth is not installed.

In [7]:
mm.play_sequence(sequence, mm.midi_synth.fluidsynth)

ImportError: fluidsynth() was called but pyfluidsynth is not installed.

In [6]:
import pyfluidsynth

ImportError: No module named pyfluidsynth

In [8]:
import fluidsynth